In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pymysql
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
host_name = "itta-crawling.c2hiebiagwyd.ap-northeast-2.rds.amazonaws.com"
username = "admin"
password = "ittaadmin"
database_name = "itta"

In [3]:
db = pymysql.connect(
    host=host_name, 
    port=3306,
    user=username,  
    passwd=password, 
    db=database_name, 
    charset='utf8'
)

In [4]:
cursor = db.cursor()
cursor.execute("set names utf8")
db.commit()

In [5]:
# 첫 번째 SQL 문 실행
cursor.execute("DROP DATABASE IF EXISTS itta;")
db.commit()

# 두 번째 SQL 문 실행
cursor.execute("CREATE DATABASE itta DEFAULT CHARSET=utf8 COLLATE=utf8_bin;")
db.commit()

# 세 번째 SQL 문 실행
cursor.execute("USE itta;")
db.commit()

# 네 번째 SQL 문 실행
cursor.execute("DROP TABLE IF EXISTS `EVENT`;")
db.commit()

# 다섯 번째 SQL 문 실행
cursor.execute("""
CREATE TABLE `EVENT` (
    event_id INT NOT NULL AUTO_INCREMENT,
    title VARCHAR(80) NOT NULL,
    date VARCHAR(50),
    price VARCHAR(50),
    location VARCHAR(50),
    address VARCHAR(50),
    type CHAR(5) NOT NULL,
    category CHAR(15) NOT NULL,
    host VARCHAR(50),
    link VARCHAR(150),
    applydate VARCHAR(50),
    thumbnail VARCHAR(300),
    PRIMARY KEY(event_id)
) DEFAULT CHARSET=utf8;
""")
db.commit()


In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html=urlopen('https://event-us.kr/search')
bs=BeautifulSoup(html,'html.parser')


In [15]:
def crawl_page(page):

    url = "https://event-us.kr/search?order=score&date=%EB%AA%A8%EB%93%A0%EB%82%A0&page="+page

    chrome_options = Options()
    # chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)

    # 웹 페이지 로드
    driver.get(url)

    # Selenium으로 페이지 스크랩
    rendered_html = driver.page_source

    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(rendered_html, 'html.parser')
    
    events=soup.findAll('a',{'class':'aspect-h-9'})
    
    cnt=0
    for event in events:
        print("----------------------------")
        if(cnt>3):
            print("충분히 업데이트 된 것 같습니다")
            return -1
        href_value = event.get('href')
        driver.get(f"https://event-us.kr/{href_value}")
        new_page_html = driver.page_source
        new_page_soup = BeautifulSoup(new_page_html, 'html.parser')

        type=new_page_soup.find('a', {'class': 'px-2 text-xs'}).text

        if(type=="강연/세미나"): 
            type="세미나"
            print(type)
        elif(type=="모임/커뮤니티" or type=="멘토링/대외활동" or type=="회의/컨벤션"):
            type="커뮤니티"
            print(type)
        elif(type=="박람회/페어"):
            type="박람회"
            print(type)
        elif(type=="박람회/페어"):
            type="박람회"
            print(type)
        elif(type=="공연/전시"):
            type="전시회"
            print(type)
        else:
            continue

        title = new_page_soup.find('div', {'class': 'text-xl'}).text[1:]
        print(title)

        cursor.execute("SELECT COUNT(*) FROM `EVENT` WHERE title = %s", (title))
        count = cursor.fetchone()[0]

        if count > 0:
            print(f"'{title}' 이미 존재..")
            cnt=cnt+1
            continue

        infoSection=new_page_soup.find('section', {'id': 'infoSection'})

        date=infoSection.dl.find_all('div')[1].dd.span.text[1:]
        print(date)

        price=infoSection.dl.find_all('div')[2].dd.span.text
        print(price)

        mapSection=new_page_soup.find('section', {'id': 'mapSection'})

        location=mapSection.dl.find_all('div')[0].dd
        if(location): 
            location=location.text
        else: 
            location=""
        print(location)


        address=mapSection.dl.find_all('div')
        if(len(address)>1):
            address=address[1].dd.div.text[1:]
        else:
            address=""
        print(address)

        category=new_page_soup.find('a', {'class': 'pr-2 text-xs'}).text
        print(category)

        host=new_page_soup.find('div',{'class':'font-bold flex flex-col'}).text[1:]
        print(host)

        link=new_page_soup.find('form',{'class':'space-y-4'}).get('action')
        link="https://event-us.kr/"+link
        print(link)

        applydate=infoSection.dl.find_all('div')[0].dd.span.text[1:]
        print(applydate)

        thumbnail=new_page_soup.find('img',{'class':'w-full rounded-md border'}).get('src')
        print(thumbnail)

        cursor.execute("""
        INSERT INTO `EVENT`(title,date,price,location,address,type,category,host,link,applydate,thumbnail)
        VALUES  ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s') 
        """%(title.replace('\'', ''),date,price.replace(',', ''),location.replace('\'', ''),address.replace('\'', ''),type,category,host,link,applydate,thumbnail))
        db.commit()
        
        return 0

In [16]:
n=0
for i in range (1, 79):
    page = str(i)
    n = crawl_page(page)
    
    if n == -1:
        print("탐색 중지")
        break

----------------------------
세미나
자원재생활동가 컨퍼런스 2023 
'자원재생활동가 컨퍼런스 2023 ' 이미 존재..
----------------------------
----------------------------
커뮤니티
초원의 밤 with 드로우앤드류 (청년 크리스천 토크 콘서트) 
'초원의 밤 with 드로우앤드류 (청년 크리스천 토크 콘서트) ' 이미 존재..
----------------------------
전시회
뮤지컬 <마리아 메리안> 
'뮤지컬 <마리아 메리안> ' 이미 존재..
----------------------------
커뮤니티
[2023 마음,클럽] 생각과 같이 사는 방법 
'[2023 마음,클럽] 생각과 같이 사는 방법 ' 이미 존재..
----------------------------
충분히 업데이트 된 것 같습니다
Exiting loop


In [ ]:
db.close()

In [ ]:
# 브라우저 닫기
driver.quit()